In [ ]:
#CMU 11747

In [1]:
import dynet as dy

In [ ]:
## Make v parameters close to 0
model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)
p_v = model.add_parameters(10)
for i in xrange(1000):
    dy.renew_cg()
    v = dy.parameter(p_v)
    v2 = dy.dot_product(v,v)
    v2.forward()
    v2.backward()
    trainer.update()

v.vec_value()

In [ ]:
## Bag of words

In [2]:
from collections import defaultdict
import time
import random
import numpy as np

In [3]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])


In [4]:
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK,w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)
print(nwords)
print(ntags)

18648
5


In [5]:
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
# # lookup for ntags
# W_sm = model.add_lookup_parameters((nwords,ntags))
# b_sm = model.add_parameters((ntags))


#with word embedding
EMBEDDING_SIZE = 64
W_emb = model.add_lookup_parameters((nwords,EMBEDDING_SIZE))
W_sm = model.add_parameters((ntags, EMBEDDING_SIZE))
b_sm = model.add_parameters((ntags))



#somewhat neural net
EMB_SIZE = 64
HID_SIZE = 64
HID_LAY = 2
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_h = [model.add_parameters((HID_SIZE, EMB_SIZE if lay == 0 else HID_SIZE)) for lay in range(HID_LAY)]
b_h = [model.add_parameters((HID_SIZE)) for lay in range(HID_LAY)]
W_sm = model.add_parameters((ntags, HID_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))    


In [7]:
# def calc_scores(words):
#     dy.renew_cg()
#     score = dy.esum([dy.lookup(W_sm,x) for x in words])
#     b_sm_exp = dy.parameter(b_sm)
#     return score + b_sm_exp


# def calc_scores(words):
#     dy.renew_cg()
#     embedding_bow = dy.esum([dy.lookup(W_emb,x) for x in words])
#     W_sm_exp = dy.parameter(W_sm)
#     b_sm_exp = dy.parameter(b_sm)
#     return W_sm_exp * embedding_bow + b_sm_exp


def calc_scores(words):
    dy.renew_cg()
    h = dy.esum([dy.lookup(W_emb, x) for x in words])
    for W_h_i, b_h_i in zip(W_h, b_h):
        h = dy.tanh( dy.parameter(W_h_i) * h + dy.parameter(b_h_i) )
    return dy.parameter(W_sm) * h + dy.parameter(b_sm)



In [8]:
for iteration in range(100):
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += loss.value()
        loss.backward()
        trainer.update()
    print("iteration %r: train loss %.4f, time %.2fs" %(iteration, train_loss/len(train), time.time()-start))
    
    testing = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
        if predict == tag:
            testing += 1
    print("iter %r: test accuracy %.4f" % (iteration, testing/len(dev)))
    

iteration 0: train loss 1.5420, time 0.64s
iter 0: test accuracy 0.3579
iteration 1: train loss 1.2572, time 0.59s
iter 1: test accuracy 0.4072
iteration 2: train loss 1.0159, time 0.62s
iter 2: test accuracy 0.4181
iteration 3: train loss 0.7987, time 0.60s
iter 3: test accuracy 0.4090
iteration 4: train loss 0.5894, time 0.59s
iter 4: test accuracy 0.4014
iteration 5: train loss 0.4308, time 0.60s
iter 5: test accuracy 0.3937
iteration 6: train loss 0.3067, time 0.75s
iter 6: test accuracy 0.3860
iteration 7: train loss 0.2285, time 0.68s
iter 7: test accuracy 0.3715
iteration 8: train loss 0.1553, time 0.70s
iter 8: test accuracy 0.3900
iteration 9: train loss 0.1081, time 0.77s
iter 9: test accuracy 0.3887
iteration 10: train loss 0.0902, time 0.85s
iter 10: test accuracy 0.3828
iteration 11: train loss 0.0630, time 0.73s
iter 11: test accuracy 0.3950
iteration 12: train loss 0.0491, time 0.63s
iter 12: test accuracy 0.3729
iteration 13: train loss 0.0417, time 0.59s
iter 13: test 